**目次**<a id='toc0_'></a>    
- 1. [科学技術計算4課題](#toc1_)    
  - 1.1. [課題04-1：LU分解の関数化](#toc1_1_)    
  - 1.2. [課題04-2：連立方程式ソルバーの実装](#toc1_2_)    
  - 1.3. [課題04-3：複数右辺をもつ行列方程式のLU分解による解法](#toc1_3_)    
  - 1.4. [課題04-4：行列式の計算](#toc1_4_)    
  - 1.5. [課題04-5：コレスキー分解の実装](#toc1_5_)    
  - 1.6. [課題04-6：対称行列の正定値性](#toc1_6_)    
  - 1.7. [課題04-7：SOR法の実装](#toc1_7_)    
  - 1.8. [課題04-4：条件数と相対誤差の見積もり](#toc1_8_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[科学技術計算4課題](#toc0_)

**各課題で共通する注意事項**

- 関数にはアノテーション（引数や返り値の型情報）と`docstring`（関数の説明文）を必ず書く．
- 実装対象の機能にはnumpyやscipyの関数を用いないこと
  - つまり`np.linalg.solve()`や`np.linalg.det()`などは用いない
- 効率的な実装のためのnumpyの機能は活用すること
  - ブロードキャストやndarrayのインデキシングやスライシングなど

## 1.1. <a id='toc1_1_'></a>[課題04-1：LU分解の関数化](#toc0_)

演習資料では「部分ピボット選択つきLU分解のアルゴリズム」を行うコードを示した．この課題では，演習資料のコードを利用して，行列 $ A $ のLU分解 $ A = PLU $ を行う関数`lu_decomposition()`を作成する．この関数は，行列 $ A $ を引数に取り，置換行列 $ P $，下三角行列 $ L $，および上三角行列 $ U $ を返す．

**やること**

1. 部分ピボット選択を用いるLU分解を実装し，$A = P L U$を満たす`P`, `L`, `U`を返す関数を作成する．
    - $P$ は置換行列．
    - $L$ は単位対角（対角成分がすべて1）の下三角行列．
    - $U$ は上三角行列．
2. `np.allclose(A, P @ L @ U)` を用いて検証する．


**関数定義**

```python
def lu_decomposition(
    A: np.ndarray,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
```

**実行手順**

1. $ n $ を2から10程度にランダムに変更し，いくつかのランダムな行列 $ A $ を生成する．
2. 実装した `lu_decomposition()` の戻り値 $P,L,U$ と，`scipy.linalg.lu(A)` の戻り値 $P_s,L_s,U_s$ を比較し，結果が同じであることを確認する．
   - $||A - P L U||$ と $||A - P_s L_s U_s||$ がともに十分小さいことを確認する．
   - $P = P_s, L = L_s, U = U_s$であるかどうかを確認する．


**注意事項**

ランダムに行列を生成すると，正則でない行列が生成される可能性がある．その場合，LU分解がうまくいかないことがあるため，エラー処理を実装する必要がある．結果が一致しない場合，バグによるものか，行列の性質によるものかを考慮せよ．



In [1]:
import numpy as np
from typing import Tuple
from tqdm.auto import tqdm
import scipy

rng = np.random.default_rng()

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
def lu_decomposition(
    A: np.ndarray,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    LU decomposition of A = PLU

    Args:
        A (np.ndarray): nxn matrix A

    Returns:
        (np.ndarray): nxn permutation matrix P
        (np.ndarray): nxn lower matrix L with 1s in diagonal
        (np.ndarray): nxn strictly upper matrix U
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    A = A.copy()

    # code here

    return P, L, U


In [3]:
for i in tqdm(range(50)):
    n = rng.integers(low=2, high=10)
    A = rng.random(size=(n, n))

    P, L, U = lu_decomposition(A)
    Psp, Lsp, Usp = scipy.linalg.lu(A)

    assert np.allclose(P, Psp), "values don't match"
    assert np.allclose(L, Lsp), "values don't match"
    assert np.allclose(U, Usp), "values don't match"


  0%|          | 0/50 [00:00<?, ?it/s]

NameError: name 'P' is not defined

## 1.2. <a id='toc1_2_'></a>[課題04-2：連立方程式ソルバーの実装](#toc0_)

上記の課題で作成した関数`lu_decomposition()`を用いて，連立方程式 $ A \boldsymbol{x} = \boldsymbol{b} $ を解く関数`solve_lu()`を実装する．

この関数は，`lu_decomposition()`の出力である置換行列$P$，下三角行列$L$，上三角行列$U$，およびベクトル$\boldsymbol{b}$を引数に取り，連立方程式の解$\boldsymbol{x}$を返す．LU分解が与えられるため，実装する機能は前進代入と後退代入である．


**やること**

1. **前進代入** $L \boldsymbol{y} = P^T \boldsymbol{b}$ で$\boldsymbol{y}$を求める．
2. **後退代入** $U \boldsymbol{x} = \boldsymbol{y}$ で$\boldsymbol{x}$を求める．
3. 上記の手順で得られた $\boldsymbol{x}$ を返す関数 `solve_lu()` を実装する．


**関数定義**

```python
def solve_lu(
    P: np.ndarray,
    L: np.ndarray,
    U: np.ndarray,
    b: np.ndarray,
) -> np.ndarray:
```

**実行手順**

1. $ n $ を2から10程度にランダムに変更し，いくつかのランダムな行列 $ A $ とベクトル $ \boldsymbol{b} $ を生成せよ．
2. 実装した関数`lu_decomposition()`と`solve_lu()`を組み合わせて得られた結果$\boldsymbol{x}$が，`np.linalg.solve()`と同じ結果を返すことを`np.allclose`を用いて確認せよ．


**注意事項**

- 内部では必ず上記で作成した `lu_decomposition(A)` を呼び出すこと
- ランダムに生成した行列が正則でない場合，連立方程式の解が存在しないことがあるため，エラーになる場合がある（この課題ではエラー処理の実装は不要である）．結果が一致しない場合は，バグによるものか，行列の性質によるものかを考慮せよ．



In [ ]:
def solve_lu(
    P: np.ndarray,
    L: np.ndarray,
    U: np.ndarray,
    b: np.ndarray,
) -> np.ndarray:
    """Solve Ax = b with PLU, LU decomposition of A = PLU

    Args:
        P (np.ndarray): nxn permutation matrix P
        L (np.ndarray): nxn lower matrix L with 1s in diagonal
        U (np.ndarray): nxn strictly upper matrix U
        b (np.ndarray): n-d rhs vector b

    Returns:
        (np.ndarray): solution x of Ax = PLUx = b
    """
    assert b.ndim == 1
    n = b.shape[0]
    assert P.shape == (n, n)
    assert L.shape == (n, n)
    assert U.shape == (n, n)
    b = b.copy()

    # code here
    # solve Ly = P^T b
    # solve Ux = y

    return x


In [ ]:
for i in tqdm(range(50)):
    n = rng.integers(low=2, high=10)
    A = rng.random(size=(n, n))
    b = rng.random(n)

    P, L, U = lu_decomposition(A)
    x_with_lu = solve_lu(P, L, U, b)

    x_numpy = np.linalg.solve(A, b)

    assert np.allclose(
        x_numpy, x_with_lu
    ), "values don't match"


## 1.3. <a id='toc1_3_'></a>[課題04-3：複数右辺をもつ行列方程式のLU分解による解法](#toc0_)

上記の関数（`lu_decomposition()`, `solve_lu()`）を用いて，行列方程式 $A X = B$ を解く関数 `solve_with_lu()` を作成する．

ここで右辺$B$は$n \times m$行列であり，$m \ge 1$である．
つまり$m=1$の場合は通常の連立一次方程式，
$m>1$の場合は$m$個の右辺ベクトルを持つ$m$個の連立一次方程式に相当する．

**やること**

1. `lu_decomposition(A)` で $A = P L U$ を求める．
2. 複数の右辺に対する前進・後退代入を同時に実行するように実装して，$X$を求める．
    - 前進代入：$L Y = P^T B$
      - 全右辺をまとめて$P^T B$で計算する
      - $L$に対して$n \times m$の$Y$全体を同時に解く
    - 後退代入：$U X = Y$
      - $U$に対して$n \times m$の$X$全体を同時に解く
3. 上記の手順で得られた$X$を返す関数`solve_with_lu(A, B)`を実装する．

**関数定義**

```python
def solve_with_lu(
    A: np.ndarray,
    B: np.ndarray,
) -> np.ndarray:
```


**実行手順**

1. $n$を2から10程度，$m$を1から10程度にランダムに変更し，いくつかのランダムな行列$A$と$B$を生成せよ．
2. 実装した関数`solve_with_lu(A, B)`の結果が，`np.linalg.solve()`と同じ結果を返すことを`np.allclose`を用いて確認せよ．


**注意事項**

- 内部では必ず上記で作成した`lu_decomposition(A)`と`solve_lu()`を呼び出すこと

In [ ]:
def solve_with_lu(
    A: np.ndarray,
    B: np.ndarray,
) -> np.ndarray:
    """solve AX = X with LU decomposition

    Args:
        A (np.ndarray): nxn matrix A
        B (np.ndarray): nxm matrix with n-d rhs vectors b as columns

    Returns:
        (np.ndarray): the solution X
    """
    assert A.ndim == 2 and B.ndim >= 1
    assert A.shape[0] == A.shape[1] == B.shape[0]
    n = A.shape[0]
    m = B.shape[1]

    # code here
    # solve Ax=b for each column b in B

    return X


In [ ]:
for i in tqdm(range(50)):
    n = rng.integers(low=2, high=10)
    m = rng.integers(low=1, high=10)

    A = rng.random(size=(n, n))
    B = rng.random(size=(n, m))

    x_with_lu = solve_with_lu(A, B)
    x_numpy = np.linalg.solve(A, B)

    assert np.allclose(
        x_numpy, x_with_lu
    ), "values don't match"


## 1.4. <a id='toc1_4_'></a>[課題04-4：行列式の計算](#toc0_)

上記で作成した関数 `lu_decomposition()` の結果$A = P L U$を用いて，行列式を計算する関数 `det_via_lu_decomposition()` を作成せよ．
（内部では必ず上記で作成した `lu_decomposition(A)` を呼び出すこと）


$A = P L U$ より，行列式は
$$\det(A) = \det(P)\,\det(L)\,\det(U)$$
である．ここで $L$ は単位下三角行列なので $\det(L)=1$であり，
$U$は上三角行列なので$\det(U)$は$U$の対角要素の積である．
したがって
$$\det(A) = \det(P)\,\prod_{i=1}^{n} U_{ii}$$
となる．

**やること**

1. $\det(P)$を計算する．この行列式は，置換の偶奇で決まる（偶置換なら$+1$，奇置換なら$-1$）．以下の `det_P(P)` を利用してよい．
2. $\prod_i U_{ii}$を計算する．ここでは，アンダーフロー・オーバーフローに配慮した実装を行う．つまり対数を用いて$\log|U_{ii}|$を計算し，符号とともに最後に$\exp$で復元する．
   - なお$U_{ii}$が負の場合にも対処するために，符号を別に計算する．
     - $\prod_{i=1}^{n} U_{ii}
       = \prod_{i=1}^{n} \mathrm{sign}(U_{ii}) \cdot \prod_{i=1}^{n} |U_{ii}|$
3. 上記の手順で得られた $\det(A) = \det(P)\,\prod_{i=1}^{n} U_{ii}$ を返す関数 `det_via_lu_decomposition()` を実装する．

**関数定義**

```python
def det_via_lu_decomposition(
    A: np.ndarray
) -> float:
```

**実行手順**

1. $n$を2から100程度にランダムに変更し，いくつかのランダムな行列$A$を生成せよ．
2. 実装した関数`det_via_lu_decomposition(A)`の結果が，`np.linalg.det()`と同じ結果を返すことを`np.isclose()`を用いて確認せよ．
3. 行列式が非常に小さい行列（対角に非常に小さい値を含む行列など）や，非常に大きい行列でも試して，`np.linalg.det()`の結果と比較する．


**注意事項**
- 内部では必ず上記で作成した `lu_decomposition(A)` を呼び出すこと
- 実装には`np.linalg.det()`を用いず，結果の比較のためだけに利用すること．

In [ ]:
def det_P(
    P: np.ndarray
) -> float:
    """det(P) of permutation matrix P

    Args:
        P (np.ndarray): nxn permutation matrix of 0 and 1

    Returns:
        float: det(P) that is either 1 or -1
    """

    assert P.ndim == 2
    assert P.shape[0] == P.shape[1]
    for p in P.ravel()[np.nonzero(P.ravel())]:
        assert np.isclose(p, 1.0)
    n = P.shape[0]
    assert np.allclose(P.sum(axis=0), np.ones(n))
    assert np.allclose(P.sum(axis=1), np.ones(n))

    ## below three lines of code are helped with ChatGPT-4o because of its complexisity

    # Extract the permutation using np.nonzero
    permutation = np.nonzero(P)[1]

    # Calculate the number of inversions in the permutation
    inversions = sum(1 for i in range(len(permutation)) for j in range(i + 1, len(permutation)) if permutation[i] > permutation[j])

    # Return 1 if inversions are even, otherwise -1
    return 1 if inversions % 2 == 0 else -1


In [ ]:
def det_lu(
    A: np.ndarray,
) -> float:
    """compute det(A)

    Args:
        A (np.ndarray): nxn square matrix

    Returns:
        (float): det(A)
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]

    P, L, U = lu_decomposition(A)

    # code here

    return 0


In [ ]:
for i in tqdm(range(50)):
    n = rng.integers(low=2, high=10)
    A = rng.random(size=(n, n))

    det_with_lu = det_lu(A)
    det_numpy = np.linalg.det(A)

    assert np.isclose(
        det_numpy, det_with_lu
    ), "values don't match"


## 1.5. <a id='toc1_5_'></a>[課題04-5：コレスキー分解の実装](#toc0_)

上記の課題で作成した関数`lu_decomposition()`を用いて，
正定値対称行列$A$を引数に取り，$A$のコレスキー分解
$$A = LL^T$$
を計算する関数`cholesky_decomposition()`を実装する（ここで，$L$は下三角行列である）．
そのために，以下の手順を考える．

正定値対称行列$A$のLU分解を
$$A = L' U$$
とする（置換行列 $P$ は単位行列となる）．ここで$L'$は単位対角の下三角行列である．
授業資料のコレスキー分解の説明において対応する式は
$$A = L' D L'^T$$
であり，したがって
$$U = D L'^T$$
である．

ここで（$L'$の対角要素がすべて1だから）$\mathrm{diag}(D) = \mathrm{diag}(D L'^T)$が成立するので，
$$D = \mathrm{diag}(\mathrm{diag}(D L'^T))$$
である．

**やること**

1. 既に実装した `lu_decomposition()` を内部で呼び出し，その返り値 $P, L, U$ を得る．
    - $D = \mathrm{diag}(\mathrm{diag}(D L'^T))$により$D$を求める．
    - $L = L' D^{1/2}$で$L$を求める．
2. 上記の手順で得られた $L$ を返す関数 `cholesky_decomposition()` を実装する．

**関数定義**

```python
def cholesky_decomposition(
    A: np.ndarray,
) -> np.ndarray:
```


**実行手順**

1. $n$を2から100程度にランダムに変更し，いくつかのランダムな行列$A$を生成せよ．
   - 正定値対称行列を生成する方法としては，$n \times n$行列 $ B $ をランダムに生成し，$A = B B^T$ または　 $A = B^T B$ とする方法がある．このようにして生成された行列は半正定値（ほとんどの場合に正定値）となる．

2. 実装した関数`cholesky_decomposition(A)`の結果が，`np.linalg.cholesky()`と同じ結果を返すことを`np.isclose()`を用いて確認せよ．


In [ ]:
n = 3
A_org = rng.random(size=(n, n))
A_org = A_org.T @ A_org
np.fill_diagonal(A_org, A_org.sum(axis=1))
print("A\n", A_org, sep="")
b_org = rng.random(n)

A = A_org.copy()
b = b_org.copy()
print("A\n", A, sep="")
print("b", b)

In [ ]:
def cholesky_decomposition(
    A: np.ndarray,
) -> np.ndarray:
    """Cholesky decomposition of A = L L^T

    Args:
        A (np.ndarray): nxn symmetric positive definite matrix A

    Returns:
        (np.ndarray): nxn lower triangular matrix L
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    assert (A == A.T).all()
    assert np.linalg.eigvalsh(A_org).min() > 0
    n = A.shape[0]
    A = A.copy()

    # code here

    return L


## 1.6. <a id='toc1_6_'></a>[課題04-6：対称行列の正定値性](#toc0_)

1. 実対称行列$A \in R^{n \times n}$の固有値を$\lambda_1, \lambda_2, \ldots, \lambda_n$（重複度を含む）とすると，
任意の実数$c \in R$に対して，行列$A + c I$の固有値が
$\lambda_1 + c, \lambda_2 + c, \ldots, \lambda_n + c$であることを示せ．


2. 実行列$A \in R^{n \times n}$に対して
    $$
    B = (A + A^T) + (\delta - \lambda_\mathrm{min}) I
    $$
    は正定値対称行列であることを示せ．
    ここで$\lambda_\mathrm{min}$は$A + A^T$の最小固有値であり，$\delta > 0$は定数である．

3. 実行列$B \in R^{n \times n}$に対して
    $$
    A = B^T B
    $$
    は半正定値対称行列であることを示せ．

## 1.7. <a id='toc1_7_'></a>[課題04-7：SOR法の実装](#toc0_)

この資料のJacobi法 `jacobi_method()` とGauss-Seidel法 `gauss_seidel()` のコードを元にして，
SOR法を関数として実装せよ．
この関数は，$ n \times n $ 係数行列 $ A $，$ n $ 次元の右辺ベクトル $ b $，および加速係数 $ \omega $ を引数として受け取り，$ n $ 次元の解ベクトル $ x $ を返すものである．

```python
def sor(
    A: np.ndarray, b: np.ndarray, omega: float
) -> np.ndarray:
```



**実行手順**

1. $ n $ を2から1000程度に変更し，いくつかのランダムな行列 $ A $ とベクトル $ b $ を生成せよ．
2. 実装した関数 `sor(A, b, omega)` が，`np.linalg.solve()` と同じ結果を与えることを確認せよ．
3. 以下の手順に従って，解の収束の様子をプロットせよ．
   1. **プロット設定**：ある固定の行列 $ A $ とベクトル $ b $ を用いて，横軸に反復回数，縦軸に誤差（例えば $ \|Ax - b\| $ のノルム）を取る．
       - **プロットの重ね合わせ**：同じグラフ上に複数のプロットを重ねて表示し，それぞれの収束の速さを視覚的に比較できるようにせよ．

   2. **加速係数の比較**：加速係数 `omega` を0.0から2.0の範囲で変更し，収束の様子をプロットで比較せよ．なお次の手順3.では，`omega`は，0.1毎，0.05毎，0.01毎など，細かく変更する事が必要になる．

   3. **Jacobi法，Gauss-Seidel法との比較**：同じ行列 $ A $ とベクトル $ b $ について，Jacobi法，Gauss-Seidel法（`omega = 1.0`のSOR法）と比較せよ．SORがJacobi法やGauss-Seidel法よりも収束が早くなるのは，`omega`の値がいくつのときかを調査せよ．

**注意事項**

- ランダムに生成した行列が収束条件を満たさない場合，SOR法や他の数値計算法が収束しなかったりエラーになる場合がある．収束条件を満たす行列を生成して検証すること（なお課題ではエラー処理の実装は不要である）．



In [ ]:
def sor(
    A: np.ndarray,
    b: np.ndarray,
    xk: np.ndarray | None = None,
    omega: float = 1.5,
    maxiter: int = 50,
    tol: float = 1e-8,
    callback: callable = None
) -> np.ndarray:
    """SOR method for solving Ax=b

    Args:
        A (np.ndarray): nxn matrix A
        b (np.ndarray): n-d vector b
        xk (np.ndarray): n-d vector of initial value x0
        omega (float): omega of SOR
        maxiter (int, optional): max iterations. Defaults to 200.
        tol (float, optional): tolerance. Defaults to 1e-8.
        callback (callable, ``callback(diff: float, norm: float)``, optional): callback function. Defaults to None.

    Returns:
        (np.ndarray): n-d vector of the solution x
    """
    assert b.ndim == 1
    n = b.shape[0]
    assert A.shape == (n, n)

    # code here

    return x


In [ ]:
from numpy.linalg import solve

for i in range(500):
    n = rng.integers(low=2, high=10)
    A = rng.random(size=(n, n))
    b = rng.random(n)

    x_mysor = sor(A, b)
    x_numpy = solve(A, b)

    assert np.isclose(
        x_numpy, x_mysor
    ), "values doesn't match"


## 1.8. <a id='toc1_8_'></a>[課題04-8：条件数と相対誤差の見積もり](#toc0_)

この課題では，誤差の大きさと条件数の関係を確認する．
そのために，線形方程式 $ A\boldsymbol{x} = \boldsymbol{b} $ に対して，非常に小さい誤差を表す乱数行列 $\Delta A$ と乱数ベクトル $\Delta \boldsymbol{b}$ を生成し，係数行列および右辺に加えた場合の解の誤差を調べる．

**やること**

1. 次の3つの場合において，解の相対誤差が不等式で予想される範囲に収まっているかを確認する．
    - 係数行列にのみ誤差を加える：
        $(A + \Delta A)(\boldsymbol{x} + \Delta\boldsymbol{x}) = \boldsymbol{b}$
    - 右辺にのみ誤差を加える：
        $A(\boldsymbol{x} + \Delta\boldsymbol{x}) = \boldsymbol{b} + \Delta \boldsymbol{b}$
    - 係数行列と右辺の両方に誤差を加える：
        $(A + \Delta A) (\boldsymbol{x} + \Delta\boldsymbol{x}) = \boldsymbol{b} + \Delta \boldsymbol{b}$
1. 得られた解の相対誤差$\frac{\| \Delta\boldsymbol{x} \|}{\| \boldsymbol{x} \|}$または$\frac{\| \Delta\boldsymbol{x} \|}{\| \boldsymbol{x} - \Delta \boldsymbol{x} \|}$を計算し，`np.linalg.cond(A)` で求めた条件数 $\mathrm{cond}(A)$ を用いて，誤差の不等式と比較する．

**実行手順**

1. ランダムに $n=5$〜$10$ 程度の正則行列 $A$ とベクトル $\boldsymbol{b}$ を生成する．
    - $A$ の要素は 1〜10 程度の範囲で生成する．
2. $\Delta A, \Delta \boldsymbol{b}$ を生成し，上記の3つの場合の近似解を求める．
    - $\Delta A$ および $\Delta \boldsymbol{b}$ の要素は $10^{-6}$ 〜 $10^{-8}$ 程度の範囲の小さな値（誤差」と呼べる程度）とする．
3. 条件数を求めて，予想される相対誤差の範囲と実際の相対誤差を比較する．
